# Limpeza, Agregação e Análise Inicial dos Dados

Os scripts abaixo realizam a limpeza e transformação dos dados. Além disso, os dados são divididos para formar as tabelas de dimensão e tabela fato.

### Importando Bibliotecas

In [1]:
import numpy as np
import pandas as pd
import time

from itertools import combinations
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

### Importando os dados

In [2]:
df = pd.read_csv('DadosBrutos/DataCoSupplyChainDataset.csv', encoding_errors="ignore", 
                 parse_dates=['order date (DateOrders)', 'shipping date (DateOrders)'])

df.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2018-02-03 22:56:00,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2018-01-18 12:27:00,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2018-01-17 12:06:00,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2018-01-16 11:45:00,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2018-01-15 11:24:00,Standard Class


## 1 - Limpeza dos Dados

##### A primeira ação é retirar os espaços em branco e parentêses do nome das colunas para facilitar as operações seguintes

In [3]:
troca_nomes_dict = {nome_antigo:nome_antigo.replace(" ", "_").replace('(','').replace(')', '') for nome_antigo in df.columns}
df_prep = df.rename(columns=troca_nomes_dict)

##### Verificação de colunas com mais de 95% de valores nulos. Essas colunas serão excluídas

In [4]:
colunas_com_nulos = []
for i in df_prep.columns:
    porcentagem_nulos = sum(df_prep[i].isna())/len(df_prep) * 100
    if porcentagem_nulos > 5:
        print("Coluna", i , "possui", str(porcentagem_nulos) + "% de valores nulos")
        colunas_com_nulos.append(i)

Coluna Order_Zipcode possui 86.23967560201419% de valores nulos
Coluna Product_Description possui 100.0% de valores nulos


In [5]:
df_sem_nulos = df_prep.drop(columns=list(colunas_com_nulos) + ["Product_Image"])

##### Verificação de colunas com valores únicos. Essas colunas serão excluídas

In [6]:
cols_sem_variancia = []
for i in df_sem_nulos.dtypes.index:
    qtd_val_unicos = len(df_sem_nulos[i].value_counts())
    if qtd_val_unicos == 1:
        print("A coluna", i, " possui como valor único: {}".format(df_sem_nulos[i].value_counts().index[0]))
        cols_sem_variancia.append(i)

A coluna Customer_Email  possui como valor único: XXXXXXXXX
A coluna Customer_Password  possui como valor único: XXXXXXXXX
A coluna Product_Status  possui como valor único: 0


In [7]:
df_sem_val_unicos = df_sem_nulos.drop(columns=cols_sem_variancia)

##### Esse dataset possui colunas com nomes diferentes, mas com dados duplicados. As colunas duplicadas serão excluídas

In [8]:
cols_redundantes = []
cols_para_retirada = []
for i in list(combinations(df_sem_val_unicos.columns, 2)):
    if all(df_sem_val_unicos[i[0]] == df_sem_val_unicos[i[1]]):
        cols_redundantes.append(i)
        print("As colunas {} e {} são iguais".format(*i))
        col_maior = i[0] if len(i[0]) > len(i[1]) else i[1]
        cols_para_retirada.append(col_maior)

print("\n Colunas que serão retiradas", cols_para_retirada)

As colunas Benefit_per_order e Order_Profit_Per_Order são iguais
As colunas Sales_per_customer e Order_Item_Total são iguais
As colunas Category_Id e Product_Category_Id são iguais
As colunas Customer_Id e Order_Customer_Id são iguais
As colunas Order_Item_Cardprod_Id e Product_Card_Id são iguais
As colunas Order_Item_Product_Price e Product_Price são iguais

 Colunas que serão retiradas ['Order_Profit_Per_Order', 'Sales_per_customer', 'Product_Category_Id', 'Order_Customer_Id', 'Order_Item_Cardprod_Id', 'Order_Item_Product_Price']


In [9]:
df_limpo = df_sem_val_unicos.drop(columns=cols_para_retirada)

## 2 - Transformação dos Dados e Análise Inicial

##### Transformação de colunas com ID's de int para string. Isso permitirá a ánalise inicial

In [10]:
df_limpo.dtypes

Type                                   object
Days_for_shipping_real                  int64
Days_for_shipment_scheduled             int64
Benefit_per_order                     float64
Delivery_Status                        object
Late_delivery_risk                      int64
Category_Id                             int64
Category_Name                          object
Customer_City                          object
Customer_Country                       object
Customer_Fname                         object
Customer_Id                             int64
Customer_Lname                         object
Customer_Segment                       object
Customer_State                         object
Customer_Street                        object
Customer_Zipcode                      float64
Department_Id                           int64
Department_Name                        object
Latitude                              float64
Longitude                             float64
Market                            

In [11]:
df_ids_str = df_limpo
df_ids_str[['Late_delivery_risk','Category_Id', 'Customer_Id', 'Department_Id', 'Order_Id', 
            'Product_Card_Id','Customer_Zipcode', 'Order_Item_Id']] =  df_ids_str[['Late_delivery_risk','Category_Id', 
                                                                                    'Customer_Id', 'Department_Id', 'Order_Id', 
                                                                                    'Product_Card_Id', 
                                                                                    'Customer_Zipcode',
                                                                                   'Order_Item_Id']].astype(str) 
df_ids_str.dtypes

Type                                   object
Days_for_shipping_real                  int64
Days_for_shipment_scheduled             int64
Benefit_per_order                     float64
Delivery_Status                        object
Late_delivery_risk                     object
Category_Id                            object
Category_Name                          object
Customer_City                          object
Customer_Country                       object
Customer_Fname                         object
Customer_Id                            object
Customer_Lname                         object
Customer_Segment                       object
Customer_State                         object
Customer_Street                        object
Customer_Zipcode                       object
Department_Id                          object
Department_Name                        object
Latitude                              float64
Longitude                             float64
Market                            

##### Criação de tabela que contém uma análise inicial dos dados

In [12]:
df_descricao = df_ids_str.describe(include='all', datetime_is_numeric=True)
df_descricao.head(20)

,Type,Days_for_shipping_real,Days_for_shipment_scheduled,Benefit_per_order,Delivery_Status,Late_delivery_risk,Category_Id,Category_Name,Customer_City,Customer_Country,...,Sales,Order_Item_Total,Order_Region,Order_State,Order_Status,Product_Card_Id,Product_Name,Product_Price,shipping_date_DateOrders,Shipping_Mode
count,180519,180519.000000,180519.000000,180519.000000,180519,180519,180519,180519,180519,180519,...,180519.000000,180519.000000,180519,180519,180519,180519,180519,180519.000000,180519,180519
unique,4,NaN,NaN,NaN,4,2,51,50,563,2,...,NaN,NaN,23,1089,9,118,118,NaN,NaN,4
top,DEBIT,NaN,NaN,NaN,Late delivery,1,17,Cleats,Caguas,EE. UU.,...,NaN,NaN,Central America,Inglaterra,COMPLETE,365,Perfect Fitness Perfect Rip Deck,NaN,NaN,Standard Class
freq,69295,NaN,NaN,NaN,98977,98977,24551,24551,66770,111146,...,NaN,NaN,28341,6722,59491,24515,24515,NaN,NaN,107752
mean,NaN,3.497654,2.931847,21.974989,NaN,NaN,NaN,NaN,NaN,NaN,...,203.772096,183.107609,NaN,NaN,NaN,NaN,NaN,141.232550,2016-06-16 05:45:23.202433024,NaN
min,NaN,0.000000,0.000000,-4274.979980,NaN,NaN,NaN,NaN,NaN,NaN,...,9.990000,7.490000,NaN,NaN,NaN,NaN,NaN,9.990000,2015-01-03 00:00:00,NaN
25%,NaN,2.000000,2.000000,7.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,119.980003,104.379997,NaN,NaN,NaN,NaN,NaN,50.000000,2015-09-25 06:59:00,NaN
50%,NaN,3.000000,4.000000,31.520000,NaN,NaN,NaN,NaN,NaN,NaN,...,199.919998,163.990005,NaN,NaN,NaN,NaN,NaN,59.990002,2016-06-15 08:32:00,NaN
75%,NaN,5.000000,4.000000,64.800003,NaN,NaN,NaN,NaN,NaN,NaN,...,299.950012,247.399994,NaN,NaN,NaN,NaN,NaN,199.990005,2017-03-04 21:29:00,NaN
max,NaN,6.000000,4.000000,911.799988,NaN,NaN,NaN,NaN,NaN,NaN,...,1999.989990,1939.989990,NaN,NaN,NaN,NaN,NaN,1999.989990,2018-02-06 22:14:00,NaN


##### Essa tabela possui as principais agregações entre os dados e será útil na conferência dos valores do dashboard do Power BI

In [13]:
df_descricao.to_csv("AnaliseInicial/Descricao_Dados.csv")
df_ids_str.to_csv("AnaliseInicial/Dados_Vendas_Limpos.csv", index=False)

In [14]:
df_ids_str.columns

Index(['Type', 'Days_for_shipping_real', 'Days_for_shipment_scheduled',
       'Benefit_per_order', 'Delivery_Status', 'Late_delivery_risk',
       'Category_Id', 'Category_Name', 'Customer_City', 'Customer_Country',
       'Customer_Fname', 'Customer_Id', 'Customer_Lname', 'Customer_Segment',
       'Customer_State', 'Customer_Street', 'Customer_Zipcode',
       'Department_Id', 'Department_Name', 'Latitude', 'Longitude', 'Market',
       'Order_City', 'Order_Country', 'order_date_DateOrders', 'Order_Id',
       'Order_Item_Discount', 'Order_Item_Discount_Rate', 'Order_Item_Id',
       'Order_Item_Profit_Ratio', 'Order_Item_Quantity', 'Sales',
       'Order_Item_Total', 'Order_Region', 'Order_State', 'Order_Status',
       'Product_Card_Id', 'Product_Name', 'Product_Price',
       'shipping_date_DateOrders', 'Shipping_Mode'],
      dtype='object')

##### Agora, com os dados limpos e com algumas transformações. Será possível prosseguir com a divisão das tabelas e mais algumas transformações

In [15]:
df_final = df_ids_str

## 3 - Divisão dos dados em tabelas de dimensão e fato

##### A função abaixo permite obter imagens do Google de forma automatizada. Será usado para obter exemplos de imagens dos produtos do conjunto de dados.
##### Em geral, essa busca automática retorna boas imagens. Para esse conjunto de dados, ocorreram poucos casos de imagens ruins
##### Essas imagens são usadas no dashboard do Power BI

In [16]:
imagens_url = {}
def obter_imagem_url (produto):
    if produto not in imagens_url:
        google_imagens = 'https://images.google.com/'

        ser = Service("chromedriver.exe")
        op = webdriver.ChromeOptions()
        op.add_argument('--headless')
        op.add_argument("--start-maximized")
        op.add_argument("--window-size=1920,1080")
        driver = webdriver.Chrome(service=ser, options=op)

        driver.get(google_imagens)
        driver.implicitly_wait(15)

        driver.find_element(By.CLASS_NAME, 'gLFyf').send_keys(produto)
        tag_encontrada = False
        tentativas = 0
        while tag_encontrada == False and tentativas < 10:
            print("Tentativa de Realizar a Pesquisa", (tentativas + 1))
            try:
                time.sleep(1)
                tag_pesquisa = driver.find_element(By.CLASS_NAME, 'Tg7LZd')
                ActionChains(driver).move_to_element(tag_pesquisa).click(tag_pesquisa).perform()
                tag_encontrada = True
            except:
                try:
                    tag_teclado = driver.find_element(By.CLASS_NAME, 'ly0Ckb')
                    ActionChains(driver).move_to_element(tag_teclado).click(tag_teclado).perform()
                    time.sleep(0.4)
                    tag_pesquisa = driver.find_element(By.CLASS_NAME, 'gNO89b')
                    ActionChains(driver).move_to_element(tag_pesquisa).click(tag_pesquisa).perform()
                    tag_encontrada = True
                except:
                    tag_encontrada = False
                    tentativas += 1
                
        
        tag_imagem = driver.find_element(By.CLASS_NAME, 'rg_i.Q4LuWd') 
        ActionChains(driver).move_to_element(tag_imagem).click(tag_imagem).perform()
                
        tag_encontrada = False
        tentativas = 0
        while tag_encontrada == False and tentativas < 10:
            print("Tentativa de Obter a Imagem", (tentativas + 1))
            try:
                time.sleep(1)
                pagina_completa = BeautifulSoup(driver.page_source, 'html.parser')
                tag_imagem_ampliada = pagina_completa.find(class_ = 'n3VNCb')
                
                url_imagem = tag_imagem_ampliada['src']
                imagens_url[produto] = url_imagem
                tag_encontrada = True
            except:
                tag_encontrada = False
                tentativas += 1
                
        url_imagem = url_imagem if tentativas < 10 else ''
        print(produto, " -> ", url_imagem[0:20])
    else:
        url_imagem = imagens_url[produto]

    return url_imagem
      

##### As colunas que possuem o nome de países desse conjunto de dados estão com vários erros na escrita
##### Através de um webscraping no Google, a função abaixo permite acertar a escrita do nome de todos os países desse conjunto de dados

In [17]:
df_final["Order_Country"].head()

0    Indonesia
1        India
2        India
3    Australia
4    Australia
Name: Order_Country, dtype: object

In [18]:
paises_corretos = {}

def traduzir_pais (valor) :
    
    if valor not in paises_corretos:
        url = 'https://www.google.com.br/'
        
        ser = Service("chromedriver.exe")
        op = webdriver.ChromeOptions()
        op.add_argument('--headless')
        op.add_argument("--start-maximized")
        op.add_argument("--window-size=1920,1080")
        
        driver = webdriver.Chrome(service=ser, options=op)  
        driver.get(url)
        driver.implicitly_wait(15)
        
        texto_pesquisa = f'{valor} pais'
        tag_digita_pesquisa = driver.find_element(By.CLASS_NAME, 'gLFyf')
        tag_digita_pesquisa.send_keys(texto_pesquisa)
        tag_digita_pesquisa.send_keys(Keys.RETURN) 
            
        pais_encontrado = False
        texto_pais = ''
        tentativas = 0
        while pais_encontrado == False and tentativas < 10:
            print("Tentativa", (tentativas + 1))
            time.sleep(1)
            pagina_resultados = BeautifulSoup(driver.page_source, features= 'html.parser')
            
            tags_resultados = pagina_resultados.find_all(class_ = 'LC20lb')
            
            for resultado in tags_resultados:
                if ' – Wikipédia, a enciclopédia livre' in resultado.text:
                    texto_pais = resultado.text.replace(' – Wikipédia, a enciclopédia livre', '').strip()
                    texto_pais = texto_pais.replace('Países Baixos', 'Irã')
                    pais_encontrado = True
                    paises_corretos[valor] = texto_pais
                    break
                    
            if pais_encontrado == False:
                tentativas += 1
        
        texto_pais = texto_pais if tentativas < 10 else valor
        print(valor, " -> ", texto_pais)
    else:
        texto_pais = paises_corretos[valor]
        
    return texto_pais


##### A função abaixo realiza a tradução para o portguês de todas as colunas desejadas. Isso é feito através de um webscraping no Google Tradutor
##### Em geral, a função funciona bem, com poucos erros. Alguns dos erros encontrados foram tratados de forma manual, dentro da própria função

In [19]:
traducoes = {}

def traduzir (valor):
    if valor not in traducoes:
        texto_fonte = valor.replace(' ', '%20')
        url = f'https://translate.google.com/?hl=pt&sl=en&tl=pt&text={texto_fonte}&op=translate'
        
        ser = Service("chromedriver.exe")
        op = webdriver.ChromeOptions()
        op.add_argument('--headless')
        driver = webdriver.Chrome(service=ser, options=op)
        
        driver.get(url)
        driver.implicitly_wait(15)
        
        traducao_realizada = False
        tentativas = 0
        while traducao_realizada == False and tentativas < 10:
            print("Tentativa", (tentativas + 1))
            time.sleep(1)
            pagina_traducao = BeautifulSoup(driver.page_source, features= 'html.parser')
            tag_traducao = pagina_traducao.find(class_ = 'ryNqvb') 
            if tag_traducao == None:
                tags_traducoes = pagina_traducao.find_all(class_ = 'HwtZe')
                if len(tags_traducoes) == 2:
                    tag_traducao = tags_traducoes[1]
                elif len(tags_traducoes) == 1:
                    tag_traducao = tags_traducoes[0]
                else:
                    tag_traducao = None
            if tag_traducao != None:
                texto_traduzido = tag_traducao.text.strip().title()
                texto_traduzido = texto_traduzido if texto_traduzido else valor
                texto_traduzido = texto_traduzido.replace('Eua','EUA').replace('Latam','América Latina').replace('Usca','América do Norte')
                texto_traduzido = texto_traduzido.replace('Forma De Pagamento','Pagamento').replace('Transferir', 'Transferência')
                traducoes[valor] = texto_traduzido
                traducao_realizada = True
            else:
                traducao_realizada = False
                tentativas += 1
        
        texto_traduzido = texto_traduzido if tentativas < 10 else valor
        print(valor, " -> ", texto_traduzido)
    else:
        texto_traduzido = traducoes[valor]
        
    return texto_traduzido

### Tabelas Dimensão

Em todos os casos, os dados serão traduzidos para o português. Além disso, podem ocorrer transformações extras usando as funções vistas acima

##### Cria a tabela dimensão de produto

In [20]:
produtos = df_final[['Product_Card_Id','Product_Name','Product_Price', 'Department_Name',  'Category_Name']]

troca_nomes_dict = {'Product_Card_Id': 'Produto_Id', 'Product_Name':'Produto_Nome', 'Product_Price': 'Produto_Preco',
                   'Department_Name': 'Departamento_Nome', 'Category_Name': 'Categoria_Nome'}

produtos = produtos.rename(columns=troca_nomes_dict)
produtos = produtos.drop_duplicates()

produtos['Produto_Imagem_URL'] = produtos['Produto_Nome'].map(obter_imagem_url)

produtos['Departamento_Nome'] = produtos['Departamento_Nome'].map(traduzir)
produtos['Categoria_Nome'] = produtos['Categoria_Nome'].map(traduzir)



Tentativa de Realizar a Pesquisa 1
Tentativa de Obter a Imagem 1
Smart watch   ->  https://a-static.mlc
Tentativa de Realizar a Pesquisa 1
Tentativa de Obter a Imagem 1
Perfect Fitness Perfect Rip Deck  ->  data:image/jpeg;base
Tentativa de Realizar a Pesquisa 1
Tentativa de Obter a Imagem 1
Under Armour Girls' Toddler Spine Surge Runni  ->  https://i5.walmartim
Tentativa de Realizar a Pesquisa 1
Tentativa de Obter a Imagem 1
Nike Men's Dri-FIT Victory Golf Polo  ->  data:image/jpeg;base
Tentativa de Realizar a Pesquisa 1
Tentativa de Obter a Imagem 1
Under Armour Men's Compression EV SL Slide  ->  https://s7d1.scene7.
Tentativa de Realizar a Pesquisa 1
Tentativa de Obter a Imagem 1
Under Armour Women's Micro G Skulpt Running S  ->  https://i.ebayimg.co
Tentativa de Realizar a Pesquisa 1
Tentativa de Obter a Imagem 1
Nike Men's Free 5.0+ Running Shoe  ->  https://m.media-amaz
Tentativa de Realizar a Pesquisa 1
Tentativa de Obter a Imagem 1
Glove It Women's Mod Oval 3-Zip Carry All Gol 

Tentativa de Realizar a Pesquisa 1
Tentativa de Obter a Imagem 1
Nike Women's Free 5.0 TR FIT PRT 4 Training S  ->  https://i.ebayimg.co
Tentativa de Realizar a Pesquisa 1
Tentativa de Obter a Imagem 1
Hirzl Women's Soffft Flex Golf Glove  ->  data:image/jpeg;base
Tentativa de Realizar a Pesquisa 1
Tentativa de Obter a Imagem 1
The North Face Women's Recon Backpack  ->  https://m.media-amaz
Tentativa de Realizar a Pesquisa 1
Tentativa de Obter a Imagem 1
Lawn mower  ->  data:image/jpeg;base
Tentativa de Realizar a Pesquisa 1
Tentativa de Obter a Imagem 1
Nike Dri-FIT Crew Sock 6 Pack  ->  https://m.media-amaz
Tentativa de Realizar a Pesquisa 1
Tentativa de Obter a Imagem 1
Nike Women's Legend V-Neck T-Shirt  ->  https://encrypted-tb
Tentativa de Realizar a Pesquisa 1
Tentativa de Obter a Imagem 1
Garmin Approach S4 Golf GPS Watch  ->  https://res.garmin.c
Tentativa de Realizar a Pesquisa 1
Tentativa de Obter a Imagem 1
insta-bed Neverflat Air Mattress  ->  data:image/jpeg;base
Tentativ

Tentativa 1
Tennis & Racquet  ->  Tênis
Tentativa 1
Fitness Accessories  ->  Acessórios Para Fitness
Tentativa 1
As Seen on  TV!  ->  Como Visto Na Tv!
Tentativa 1
Golf Balls  ->  Bolas De Golf
Tentativa 1
Tentativa 2
Strength Training  ->  Treinamento De Força
Tentativa 1
Tentativa 2
Children's Clothing  ->  Roupas Infantis
Tentativa 1
Tentativa 2
Lacrosse  ->  Lacrosse
Tentativa 1
Baby   ->  Bebê
Tentativa 1
Fishing  ->  Pescaria
Tentativa 1
Books   ->  Livros
Tentativa 1
DVDs  ->  Dvds
Tentativa 1
Tentativa 2
CDs   ->  Cds
Tentativa 1
Tentativa 2
Garden  ->  Jardim
Tentativa 1
Tentativa 2
Hockey  ->  Hóquei
Tentativa 1
Tentativa 2
Pet Supplies  ->  Suprimentos Para Animais De Estimação
Tentativa 1
Health and Beauty  ->  Saúde E Beleza
Tentativa 1
Music  ->  Música
Tentativa 1
Video Games  ->  Videogames
Tentativa 1
Golf Gloves  ->  Luvas De Golfe
Tentativa 1
Golf Bags & Carts  ->  Sacos De Golfe
Tentativa 1
Tentativa 2
Tentativa 3
Golf Shoes  ->  Sapatos De Golfe
Tentativa 1
Golf Ap

##### Cria a tabela dimensão de cliente

In [21]:
def preenche_nome_completo (lista):
    if lista[1] == None or type(lista[1]) is not str:
        nome_completo = lista[0]
    else:
        if len(lista[1].strip()) == 0:
            nome_completo = lista[0]
        else:
            nome_completo = lista[0] + ' ' + lista[1]

    return nome_completo

In [22]:
clientes = df_final[['Customer_Id', 'Customer_City', 'Customer_Country', 'Customer_Fname', 'Customer_Lname', 
                     'Customer_Segment', 'Customer_State', 'Customer_Street', 'Customer_Zipcode', 'Latitude', 'Longitude']]

clientes['Cliente_Nome_Completo'] = clientes.loc[:,('Customer_Fname','Customer_Lname')].apply(lambda x : preenche_nome_completo(x), axis=1)
clientes = clientes[['Customer_Id', 'Cliente_Nome_Completo', 'Customer_Segment', 'Customer_Country', 'Customer_State', 
                     'Customer_City', 'Customer_Street', 'Customer_Zipcode', 'Latitude', 'Longitude']]

troca_nomes_dict = {'Customer_Id':'Cliente_Id', 'Customer_City': 'Cliente_Cidade', 'Customer_Country': 'Cliente_Pais',
                   'Customer_Segment': 'Cliente_Segmento', 'Customer_State': 'Cliente_Estado', 'Customer_Street': 'Cliente_Rua',
                   'Customer_Zipcode': 'Cliente_CEP', 'Latitude': 'Cliente_Latitude', 'Longitude': 'Cliente_Longitude'}

clientes = clientes.rename(columns=troca_nomes_dict)
clientes = clientes.drop_duplicates()

traducoes = {}
clientes['Cliente_Segmento'] = clientes['Cliente_Segmento'].map(traduzir)
clientes['Cliente_Pais'] = clientes['Cliente_Pais'].map(traduzir_pais)

C:\Users\Luann\AppData\Local\Temp\ipykernel_5388\649134544.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clientes['Cliente_Nome_Completo'] = clientes.loc[:,('Customer_Fname','Customer_Lname')].apply(lambda x : preenche_nome_completo(x), axis=1)


Tentativa 1
Tentativa 2
Tentativa 3
Tentativa 4
Consumer  ->  Consumidor
Tentativa 1
Home Office  ->  Escritório Em Casa
Tentativa 1
Tentativa 2
Corporate  ->  Corporativo
Tentativa 1
Puerto Rico  ->  Porto Rico
Tentativa 1
EE. UU.  ->  Estados Unidos


##### Cria a tabela dimensão de pedidos

In [23]:
pedidos = df_final[['Order_Id', 'Type', 'Market', 'Order_Region', 'Order_Country', 'Order_State', 'Order_City', 
                    'Shipping_Mode', 'Days_for_shipping_real', 'Days_for_shipment_scheduled', 'Late_delivery_risk' ]]

troca_nomes_dict = {'Order_Id': 'Pedido_Id', 'Order_Region': 'Pedido_Regiao', 'Late_delivery_risk': 'Pedido_Risco_Atraso', 
                    'Days_for_shipping_real': 'Pedido_Dias_Envio_Real',
                    'Days_for_shipment_scheduled': 'Pedido_Dias_Envio_Agendado', 'Order_City': 'Pedido_Cidade',
                    'Order_Country': 'Pedido_Pais','Shipping_Mode': 'Pedido_Modo_Envio', 'Market': 'Pedido_Mercado',
                    'Type': 'Pedido_Forma_Pagamento', 'Order_State': 'Pedido_Estado'}

pedidos = pedidos.rename(columns=troca_nomes_dict)
pedidos = pedidos.drop_duplicates()

traducoes = {}
pedidos['Pedido_Mercado'] = pedidos['Pedido_Mercado'].map(traduzir)
pedidos['Pedido_Regiao'] = pedidos['Pedido_Regiao'].map(traduzir)
pedidos['Pedido_Modo_Envio'] = pedidos['Pedido_Modo_Envio'].map(traduzir)
pedidos['Pedido_Forma_Pagamento'] = pedidos['Pedido_Forma_Pagamento'].map(traduzir)
pedidos['Pedido_Pais'] = pedidos['Pedido_Pais'].map(traduzir_pais)

Tentativa 1
Pacific Asia  ->  Pacífico Ásia
Tentativa 1
USCA  ->  América do Norte
Tentativa 1
Africa  ->  África
Tentativa 1
Europe  ->  Europa
Tentativa 1
LATAM  ->  América Latina
Tentativa 1
Southeast Asia  ->  Sudeste Da Ásia
Tentativa 1
South Asia  ->  Sul Da Asia
Tentativa 1
Tentativa 2
Oceania  ->  Oceânia
Tentativa 1
Eastern Asia  ->  Ásia Oriental
Tentativa 1
Tentativa 2
Tentativa 3
West Asia  ->  Oeste Da Ásia
Tentativa 1
West of USA   ->  Oeste Dos EUA
Tentativa 1
Tentativa 2
US Center   ->  Us Center
Tentativa 1
West Africa  ->  África Ocidental
Tentativa 1
Tentativa 2
Central Africa  ->  África Central
Tentativa 1
North Africa  ->  Norte Da África
Tentativa 1
Tentativa 2
Western Europe  ->  Europa Ocidental
Tentativa 1
Northern Europe  ->  Norte Da Europa
Tentativa 1
Central America  ->  América Central
Tentativa 1
Tentativa 2
Caribbean  ->  Caribe
Tentativa 1
Tentativa 2
South America  ->  América Do Sul
Tentativa 1
Tentativa 2
East Africa  ->  Este De África
Tentativa 1

Tentativa 1
Repblica de Gambia  ->  Gâmbia
Tentativa 1
Botsuana  ->  Botswana
Tentativa 1
Armenia  ->  Arménia
Tentativa 1
Guinea Ecuatorial  ->  Guiné Equatorial
Tentativa 1
Kuwait  ->  Kuwait
Tentativa 1
Butn  ->  Butão
Tentativa 1
Chad  ->  Chade
Tentativa 1
Serbia  ->  Sérvia
Tentativa 1
Shara Occidental  ->  Saara Ocidental


##### Cria a tabela dimensão tempo

In [24]:
comeco_DateOrder = df_final['order_date_DateOrders'].min()
comeco_ShippingDateOrder = df_final['shipping_date_DateOrders'].min()

comeco = comeco_DateOrder if comeco_DateOrder < comeco_ShippingDateOrder else comeco_ShippingDateOrder

fim_DateOrder = df_final['order_date_DateOrders'].max()
fim_ShippingDateOrder = df_final['shipping_date_DateOrders'].max()

fim = fim_DateOrder if fim_DateOrder > fim_ShippingDateOrder else fim_ShippingDateOrder
 

dim_tempo = pd.DataFrame({"Data_completa": pd.date_range(comeco, fim, freq='min')})
dim_tempo["Dia"] = dim_tempo['Data_completa'].dt.day

dim_tempo["Dia_Semana_Num"] = dim_tempo['Data_completa'].dt.dayofweek

dim_tempo["Dia_Semana"] = dim_tempo['Data_completa'].dt.strftime("%A")
dim_tempo["Dia_Semana"] = dim_tempo["Dia_Semana"].map(traduzir)

dim_tempo["Trimestre"] = dim_tempo['Data_completa'].dt.quarter
dim_tempo["Mes"] = dim_tempo['Data_completa'].dt.month

dim_tempo["Nome_Mes"] = dim_tempo['Data_completa'].dt.month_name()
dim_tempo["Nome_Mes"] = dim_tempo["Nome_Mes"].map(traduzir)

dim_tempo["Ano"] = dim_tempo['Data_completa'].dt.year
dim_tempo["Hora"] = dim_tempo['Data_completa'].dt.hour
dim_tempo["Minuto"] = dim_tempo['Data_completa'].dt.minute
dim_tempo["Tempo_Id"] = range(1,(dim_tempo.shape[0] + 1))

dim_tempo.loc[dim_tempo["Mes"] == 3,"Nome_Mes"] = "Março"
dim_tempo.loc[dim_tempo["Mes"] == 5, "Nome_Mes"] = "Maio"


Tentativa 1
Thursday  ->  Quinta-Feira
Tentativa 1
Friday  ->  Sexta-Feira
Tentativa 1
Saturday  ->  Sábado
Tentativa 1
Sunday  ->  Domingo
Tentativa 1
Monday  ->  Segunda-Feira
Tentativa 1
Tuesday  ->  Terça-Feira
Tentativa 1
Wednesday  ->  Quarta-Feira
Tentativa 1
January  ->  Janeiro
Tentativa 1
February  ->  Fevereiro
Tentativa 1
March  ->  Marchar
Tentativa 1
April  ->  Abril
Tentativa 1
May  ->  Poderia
Tentativa 1
Tentativa 2
Tentativa 3
June  ->  Junho
Tentativa 1
July  ->  Julho
Tentativa 1
August  ->  Agosto
Tentativa 1
September  ->  Setembro
Tentativa 1
October  ->  Outubro
Tentativa 1
November  ->  Novembro
Tentativa 1
December  ->  Dezembro


### Tabela Fato

Criação da tabela fato de itens do pedido. Essa tabela possui refernência a todas as tabelas de dimensão

In [25]:
itens_pedido = df_final[['Order_Id', 'Product_Card_Id', 'Customer_Id', 'order_date_DateOrders', 'shipping_date_DateOrders',
                         'Order_Item_Quantity', 'Sales', 'Order_Item_Total', 'Order_Item_Discount',  
                         'Order_Item_Discount_Rate', 'Benefit_per_order', 'Order_Item_Profit_Ratio', 'Order_Status',
                         'Delivery_Status' ]]

troca_nomes_dict = {'Order_Id': 'Pedido_Id', 'Product_Card_Id': 'Produto_Id', 'Customer_Id': 'Cliente_Id', 
                    'Order_Item_Quantity': 'Item_Quantidade', 'Sales': 'Item_Valor_Total', 'Order_Item_Discount': 'Item_Desconto',
                   'Order_Item_Discount_Rate': 'Item_Desconto_Taxa', 'Benefit_per_order': 'Item_Lucro', 
                    'Order_Item_Profit_Ratio': 'Item_Lucro_Taxa', 'Order_Item_Total': 'Item_Valor_Pago',
                   'Order_Status': 'Pedido_Status', 'Delivery_Status': 'Pedido_Status_Entrega'}

itens_pedido = itens_pedido.rename(columns=troca_nomes_dict)

itens_pedido['Pedido_Status_Entrega'] = itens_pedido['Pedido_Status_Entrega'].map(traduzir)
itens_pedido['Pedido_Status'] = itens_pedido['Pedido_Status'].map(traduzir)

Tentativa 1
Advance shipping  ->  Envio Antecipado
Tentativa 1
Late delivery  ->  Entrega Tardia
Tentativa 1
Tentativa 2
Tentativa 3
Shipping on time  ->  Envio A Tempo
Tentativa 1
Shipping canceled  ->  Envio Cancelado
Tentativa 1
COMPLETE  ->  Completo
Tentativa 1
PENDING  ->  Pendente
Tentativa 1
Tentativa 2
CLOSED  ->  Fechado
Tentativa 1
PENDING_PAYMENT  ->  Pagamento Pendente
Tentativa 1
Tentativa 2
CANCELED  ->  Cancelado
Tentativa 1
PROCESSING  ->  Em Processamento
Tentativa 1
Tentativa 2
Tentativa 3
Tentativa 4
SUSPECTED_FRAUD  ->  Suspeita De Fraude
Tentativa 1
ON_HOLD  ->  Em Espera
Tentativa 1
PAYMENT_REVIEW  ->  Revisão De Pagamento


##### Operações de junção para obter a referência da tabela dimensão tempo

In [26]:
df_teste = dim_tempo.merge(itens_pedido, how='inner', right_on= 'order_date_DateOrders', left_on= 'Data_completa')
itens_pedido = df_teste[['Pedido_Id', 'Produto_Id', 'Cliente_Id', 'Tempo_Id','shipping_date_DateOrders', 
                         'Item_Quantidade', 'Item_Valor_Total', 'Item_Valor_Pago', 'Item_Desconto', 'Item_Desconto_Taxa',
                         'Item_Lucro', 'Item_Lucro_Taxa', 'Pedido_Status', 'Pedido_Status_Entrega']]
itens_pedido = itens_pedido.rename(columns={'Tempo_Id':'Pedido_Id_Data'})

In [27]:
df_teste = dim_tempo.merge(itens_pedido, how='inner', right_on= 'shipping_date_DateOrders', left_on= 'Data_completa')
itens_pedido = df_teste[['Pedido_Id', 'Produto_Id', 'Cliente_Id', 'Pedido_Id_Data','Tempo_Id', 
                         'Item_Quantidade', 'Item_Valor_Total', 'Item_Valor_Pago', 'Item_Desconto', 'Item_Desconto_Taxa',
                         'Item_Lucro', 'Item_Lucro_Taxa', 'Pedido_Status', 'Pedido_Status_Entrega']]
itens_pedido = itens_pedido.rename(columns={'Tempo_Id':'Pedido_Id_Envio_Data'})

##### Salva os dados que serão usados no Power BI

In [28]:
produtos.to_csv("Dados/Produtos.csv", index=False)
clientes.to_csv("Dados/Clientes.csv", index=False)
pedidos.to_csv("Dados/Pedidos.csv", index=False)
dim_tempo.to_csv("Dados/Dim_Tempo.csv", index=False)
itens_pedido.to_csv("Dados/Itens_Pedidos.csv", index=False)

## 4 - Considerações Finais

- Quase todos os dados foram traduzidos para o português. Algumas das poucas exceções são o nome dos produtos e nome de cidades.
- Existem clientes com boa parte dos dados iguais (nome completo, estado, dentre outros), mas com Id's diferentes. Dentro da tabela, esses clientes foram considerados como diferentes. Porém, nas visualizações do Power BI serão considerados como clientes iguais
- Existem pedidos com prejuízo, pedidos com lucro que foram cancelados, dentre outras situações especiais. Nesses casos, não ocorreram mudanças
- Os scripts de tradução funcionam bem para a maioria dos dados. Porém, em alguns casos podem ocorrer traduções esquisitas
- O scripts de tradução, em geral, são bem lentos e podem não funcionar por diversas razões. Além disso, podem exigir manutenção constante